<a href="https://colab.research.google.com/github/OSegun/Zummit-Africa-ML-AL-Projects/blob/main/Business_Review_Sentimental_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Review Sentimental Analyser

Our study cas or use is to provide a sentimental analysis that helps to review customers comments or feedback on the rating of 1 to 5.

We use an NLP bert pretrained model which can found from the link here https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment and https://metatext.io/models/nlptown-bert-base-multilingual-uncased-sentiment we scrape the comments from businesses website with beautiful soup and convert it to a dataframe which is then feed into the model to provide the ratings of scale 1 to 5 on the business.

In [34]:
#!pip install transformers

In [35]:
#Importing all dependences
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [36]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Instansiating the NLP bert model for use into our tokenizer
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [37]:
tokens = tokenizer.encode('You built a great empire an I do hope there are visionaries to sustain it', return_tensors='pt')

In [38]:
tokens[0]

tensor([  101, 10855, 12321,   143, 11838, 13359, 10144,   151, 10154, 18763,
        10768, 10320, 17584, 30966, 10114, 10877, 41722, 10197,   102])

In [39]:
result = model(tokens)

In [40]:
result.logits

tensor([[-1.9258, -1.7160, -0.3581,  1.2074,  2.1254]],
       grad_fn=<AddmmBackward0>)

In [41]:
int(torch.argmax(result.logits))+1

5

In [42]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

### Evaluation With Business Pages

In [43]:
# Scraping the Jumia website with product about 10 reviews from the site
r = requests.get('https://www.jumia.com.ng/catalog/productratingsreviews/sku/GE779EA1BY9XTNAFAMZ/')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('-pvs')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [44]:
reviews

['Long lasting battery and smooth working experience',
 'Perfect tool',
 'Impressed by the ability and functions and also the beauty.. worth the price',
 'A must have, but handle it gently and don’t let it fall.',
 'It’s a nice product, working well for now and it lasts long before you have to charge it. Only ick is that the white gets dirty quickly so you might wanna get the black color.',
 "It sleek on palm and it's noiseless when using it.",
 "It's slim. It's slick. It's perfect!\nIt's very comfortable and ergonomic to use. One charge can go for a week. That's quite impressive.",
 'It’s so slim, sleek, noiseless. You just need to charge it once every few days (I use mine all day to work). It’s dope dope',
 'its quit okay and nice',
 'Nice product']

In [45]:
df1 = pd.DataFrame(np.array(reviews), columns=['review']) # Converting it to pandas dataframe

In [46]:
df1['review'].iloc[1]

'Perfect tool'

In [47]:
df1.shape

(10, 1)

In [48]:
# Scraping a business on the yelp website of about 10 reviews
r = requests.get('https://www.yelp.com/biz/dennys-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('comment.')
result1 = soup.find_all('p', {'class':regex})
review1 = [result.text for result in result1]

In [49]:
sentiment_score(df1['review'].iloc[0])

5

In [50]:
df1['sentiment'] = df1['review'].apply(lambda x: sentiment_score(x[:512]))

In [51]:
df1.head()

,review,sentiment
0,Long lasting battery and smooth working experi...,5
1,Perfect tool,5
2,Impressed by the ability and functions and als...,5
3,"A must have, but handle it gently and don’t le...",4
4,"It’s a nice product, working well for now and ...",4


In [52]:
review1

['I had Jordan as my waiter. He was absolutely great! He was very welcoming, very enthusiastic, funny and held the best quality service.',
 "Tbh, I haven't eat at Denny's in ages. My mom and I decided to eat dinner here after our mini-shopping spree for Christmas. The ambiance has changed significantly but it brings me back to my childhood of eating Denny's at this particular location. I'm glad they're able to survive and thrive during the pandemic. It wasn't a busy and were seated immediately. We were able to choose our spot and sat closer to the kitchen and hallway to the restroom. I ordered the Plate Lickin' Fried Chicken with sides of red skin potatoes and steamed vegetables. My mom ordered the country fried steak with mashed potatoes and a side salad. Since we were in the mood for dinner but still wanted a small taste of breakfast, we ordered a side stack of pancakes. (If you don't want a full breakfast plate, just make sure to read carefully and look for the sides on the lower pa

In [53]:
df2 = pd.DataFrame(np.array(review1), columns=['review']) # Converting it to pandas dataframe

In [54]:
df2['review'].iloc[1]

"Tbh, I haven't eat at Denny's in ages. My mom and I decided to eat dinner here after our mini-shopping spree for Christmas. The ambiance has changed significantly but it brings me back to my childhood of eating Denny's at this particular location. I'm glad they're able to survive and thrive during the pandemic. It wasn't a busy and were seated immediately. We were able to choose our spot and sat closer to the kitchen and hallway to the restroom. I ordered the Plate Lickin' Fried Chicken with sides of red skin potatoes and steamed vegetables. My mom ordered the country fried steak with mashed potatoes and a side salad. Since we were in the mood for dinner but still wanted a small taste of breakfast, we ordered a side stack of pancakes. (If you don't want a full breakfast plate, just make sure to read carefully and look for the sides on the lower page of the menu). They also have Coke Zero in case you still want soda but calorie free, with refills. Props to Ryan for his above and beyond

In [55]:
df2.shape

(10, 1)

In [56]:
sentiment_score(df2['review'].iloc[1])

4

In [57]:
df2['sentiment'] = df2['review'].apply(lambda x: sentiment_score(x[:512]))

In [58]:
df2.head()

,review,sentiment
0,I had Jordan as my waiter. He was absolutely g...,5
1,"Tbh, I haven't eat at Denny's in ages. My mom ...",4
2,"As one of the only 24 hour open places in SF, ...",4
3,Last night at 1am to pick my brother up. My wi...,5
4,Good ol' diner food. Denny's is a nostalgic pl...,4
